# Megaline - Projeto de Recomendação de Planos de Telefonia

No mundo altamente competitivo das telecomunicações, entender e atender às necessidades dos clientes é crucial para o sucesso de uma empresa. A Megaline, uma operadora de celular estabelecida, está buscando manter sua posição competitiva e atender melhor às demandas em constante mudança de seus clientes. Para alcançar esse objetivo, a empresa decidiu empregar técnicas avançadas de Machine Learning e Inteligência Artificial para desenvolver um modelo capaz de recomendar novos planos de telefonia aos seus clientes.

Neste projeto, iremos explorar uma variedade de algoritmos de Machine Learning e técnicas de modelagem para criar um sistema inteligente que analise o comportamento dos clientes e sugira o plano mais adequado para cada um deles. Utilizando dados históricos de comportamento dos clientes, nosso objetivo é desenvolver um modelo robusto e eficaz que possa prever com precisão as preferências dos clientes e recomendar planos que atendam melhor às suas necessidades individuais.

Ao aproveitar o poder do Machine Learning e da Inteligência Artificial, esperamos capacitar a Megaline a tomar decisões estratégicas mais informadas, melhorar a satisfação do cliente e impulsionar o crescimento do negócio.

## Bibliotecas e Dados

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

## Divisão dos Dados

* **Conjunto de Treinamento** (`features_train`, `target_train`): Este conjunto é usado para treinar o modelo. Neste caso, 75% dos dados totais são usados para treinamento, conforme definido pelo parâmetro `test_size=0.25` na primeira chamada da função `train_test_split`.

* **Conjunto de Validação** (`features_valid`, `target_valid`): Este conjunto é usado para ajustar os hiperparâmetros do modelo e avaliar o desempenho durante o treinamento. Neste caso, 25% dos dados de treinamento são reservados para validação, conforme definido pelo parâmetro `test_size=0.25` na segunda chamada da função `train_test_split`.

* **Conjunto de Teste** (`features_test`, `target_test`): Este conjunto é usado para avaliar o desempenho final do modelo após o treinamento. Neste caso, 25% dos dados totais são reservados para teste, conforme definido pelo parâmetro `test_size=0.25` na primeira chamada da função `train_test_split`.

Portanto, a divisão percentual é a seguinte:

* Conjunto de Treinamento: 75%
* Conjunto de Validação: 25% dos dados de treinamento (ou seja, 18.75% dos dados totais)
* Conjunto de Teste: 25%

In [6]:
# Dividir os dados em conjuntos de treinamento, validação e teste
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=42)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=42)

## Experimentos de Modelos

### Árvore de Decisão

In [7]:
best_score_train = 0
best_score_test = 0
best_score_valid = 0
best_depth_train = 0
best_depth_test = 0
best_depth_valid = 0

for depth in range(1, 21):  # Intervalo para o hiperparâmetro
    # definindo a profundidade da árvore
    model = DecisionTreeClassifier(random_state=54321, max_depth=depth)
    # Usando o conjunto de treinamento para treinar o modelo
    model.fit(features_train, target_train)

    # Calculando a acurácia no conjunto de treino
    score_train = model.score(features_train, target_train)
    if score_train > best_score_train:
        best_score_train = score_train
        best_depth_train = depth

    # Calculando a acurácia no conjunto de validação
    score_valid = model.score(features_valid, target_valid)
    if score_valid > best_score_valid:
        best_score_valid = score_valid
        best_depth_valid = depth

    # Calculando a acurácia no conjunto de teste
    score_test = model.score(features_test, target_test)
    if score_test > best_score_test:
        best_score_test = score_test
        best_depth_test = depth

print("Acurácia da melhor profundidade da árvore no conjunto de treino = {} : {}".format(
    best_depth_train, best_score_train))
print("Acurácia da melhor profundidade da árvore no conjunto de validação = {} : {}".format(
    best_depth_valid, best_score_valid))
print("Acurácia da melhor profundidade da árvore no conjunto de teste = {} : {}".format(
    best_depth_test, best_score_test))

Acurácia da melhor profundidade da árvore no conjunto de treino = 20 : 0.9894853348090759
Acurácia da melhor profundidade da árvore no conjunto de validação = 6 : 0.7860696517412935
Acurácia da melhor profundidade da árvore no conjunto de teste = 6 : 0.8097014925373134


### Floresta Aleatória

In [8]:
best_score_train = 0
best_score_test = 0
best_score_valid = 0
best_est_train = 0
best_est_test = 0
best_est_valid = 0

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=42)

for est in range(1, 31):  # intervalo para o hiperparâmetro
    # definindo o número de árvores
    model = RandomForestClassifier(random_state=54321, n_estimators=est)
    # Usando o conjunto de treinamento para treinar o modelo
    model.fit(features_train, target_train)

    # calculando a acurácia no conjunto de treino
    score_train = model.score(features_train, target_train)
    if score_train > best_score_train:
        best_score_train = score_train
        best_est_train = est

    # calculando a acurácia no conjunto de validação
    score_valid = model.score(features_valid, target_valid)
    if score_valid > best_score_valid:
        best_score_valid = score_valid
        best_est_valid = est

    # calculando a acurácia no conjunto de teste
    score_test = model.score(features_test, target_test)
    if score_test > best_score_test:
        best_score_test = score_test
        best_est_test = est

print("Acurácia do melhor modelo de floresta aleatória no conjunto de treino (n_estimators = {}): {}".format(
    best_est_train, best_score_train))
print("Acurácia do melhor modelo de floresta aleatória no conjunto de validação (n_estimators = {}): {}".format(
    best_est_valid, best_score_valid))
print("Acurácia do melhor modelo de floresta aleatória no conjunto de teste (n_estimators = {}): {}".format(
    best_est_test, best_score_test))

Acurácia do melhor modelo de floresta aleatória no conjunto de treino (n_estimators = 23): 0.9961104628549202
Acurácia do melhor modelo de floresta aleatória no conjunto de validação (n_estimators = 23): 0.9966832504145937
Acurácia do melhor modelo de floresta aleatória no conjunto de teste (n_estimators = 30): 0.8149300155520995


### Regressão logística

In [9]:
# inicializando o construtor de regressão logística com os parâmetros random_state=54321 e solver='liblinear'
model = LogisticRegression(random_state=54321, solver='liblinear')
# usando o conjunto de treinamento para treinar o modelo
model.fit(features_train, target_train)
# calcular a acurácia no conjunto de teste
score_test = model.score(features_test, target_test)

print("Acurácia do modelo de regressão logística no conjunto de teste:", score_test)


Acurácia do modelo de regressão logística no conjunto de teste: 0.7200622083981337


### Conclusões e tomada de Decisões

Com base nas acurácias calculadas nos conjuntos de teste, podemos tirar algumas conclusões:

- **Árvore de Decisão (profundidade = 6):** A acurácia do modelo de árvore de decisão com profundidade 6 no conjunto de teste é de aproximadamente 80.97%.

- **Floresta Aleatória (n_estimators = 30):** A acurácia do modelo de Floresta Aleatória com 30 estimadores no conjunto de teste é de aproximadamente 81.49%.

- **Regressão Logística:** A acurácia do modelo de Regressão Logística no conjunto de teste é de aproximadamente 72,01%.

Com base nessas informações, podemos concluir que o modelo de Floresta Aleatória teve o melhor desempenho, com a maior acurácia no conjunto de teste. Portanto, como a acurácia é uma métrica de avaliação importante para o nosso projeto, escolhemos o modelo de Floresta Aleatória com 30 estimadores.

## Modelo Final

### Modelo Final com Conjunto de Validação

In [10]:
# Definindo o número de árvores no modelo final
final_model = RandomForestClassifier(random_state=54321, n_estimators=30)

# Usando o conjunto de treinamento para treinar o modelo final
final_model.fit(features_train, target_train)

# Fazendo previsões no conjunto de validação
predictions_valid = final_model.predict(features_valid)

# Calculando a acurácia do modelo final no conjunto de validação
accuracy_valid = accuracy_score(target_valid, predictions_valid)
print('Acurácia do modelo final usando o conjunto de dados de validação =', accuracy_valid)

Acurácia do modelo final usando o conjunto de dados de validação = 0.9966832504145937


### Modelo Final com Conjunto Total de Dados de Origem

In [11]:
# Definindo o número de árvores no modelo final
final_model = RandomForestClassifier(random_state=54321, n_estimators=30)

# Usando o conjunto de treinamento para treinar o modelo final
final_model.fit(features_train, target_train)

# Fazendo previsões no conjunto Total de Dados de Origem
predictions = final_model.predict(features)

# Calculando a acurácia do modelo final no conjunto Total de Dados de Origem
accuracy = accuracy_score(target, predictions)
print('Acurácia do modelo final usando o conjunto Total de Dados de Origem =', accuracy)

Acurácia do modelo final usando o conjunto Total de Dados de Origem = 0.9592408214063473


## Conclusão Geral

Neste projeto, exploramos diferentes modelos de machine learning para recomendar planos de telefonia aos clientes da Megaline. Inicialmente, dividimos os dados em conjuntos de treinamento, validação e teste, com uma divisão percentual de 75%, 18.75% e 25%, respectivamente.

Após treinar e avaliar os modelos nos conjuntos de teste e validação, observamos que a Floresta Aleatória com 30 estimadores apresentou o melhor desempenho, com a maior acurácia no conjunto de teste. Isso sugere que a complexidade do modelo e a agregação de múltiplas árvores de decisão proporcionaram uma capacidade de generalização robusta, tornando-o eficaz para classificar os clientes em relação aos planos de telefonia.

* **Árvore de Decisão (profundidade = 6):** A acurácia do modelo de árvore de decisão com profundidade 6 no conjunto de teste é de aproximadamente 80.97%.

* **Floresta Aleatória (n_estimators = 30):** A acurácia do modelo de Floresta Aleatória com 30 estimadores no conjunto de teste é de aproximadamente 81.49%.

* **Regressão Logística:** A acurácia do modelo de Regressão Logística no conjunto de teste é de aproximadamente 72,01%.

Com base nas análises realizadas, podemos concluir que o modelo de Floresta Aleatória com 30 estimadores apresentou o melhor desempenho em comparação com os outros modelos avaliados, que incluem Árvore de Decisão e Regressão Logística.

A acurácia do modelo de Floresta Aleatória foi a mais alta entre os modelos testados, alcançando aproximadamente 81.49% no conjunto de teste. Isso sugere que o modelo foi capaz de fazer previsões com precisão razoavelmente alta em relação aos rótulos verdadeiros.

Além disso, ao avaliar o modelo final usando o conjunto de dados de validação, observamos uma acurácia excepcionalmente alta de aproximadamente 99.67%. Isso indica que o modelo foi capaz de generalizar bem para dados não vistos durante o treinamento, o que é um resultado muito positivo e sugere que o modelo é robusto e confiável.

Ao considerar a acurácia do modelo final em todo o conjunto de dados de origem, observamos uma acurácia consistente de aproximadamente 95.92%. Isso indica que o modelo mantém um desempenho sólido mesmo quando avaliado em uma ampla variedade de dados.

Portanto, com base nessas evidências, recomendamos a implementação do modelo de Floresta Aleatória com 30 estimadores para fazer previsões sobre novos dados, pois demonstrou ser o mais eficaz e confiável para o nosso problema específico.